### Importando Bibliotecas

In [236]:
#Importação das Bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

### Carregando Base de Ddados

In [237]:
#Importando base de dados
#Dataframe de treinamento
df_train = pd.read_csv('train.csv')
#Dataframe de teste
df_test = pd.read_csv('test.csv')

### Pré-Processamento

In [238]:
#Exibindo as 2 primeiras linhas
df_train.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [239]:
#Estatísticas
df_train.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [240]:
#Verificando valores nulos
df_train.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

In [241]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [242]:
#Adicionando uma nova coluna e renomeando 
df_train.insert(1, 'month', '')
df_train.insert(2, 'hours', '')

df_train = df_train.rename(columns = {'datetime': 'date'})


In [243]:
#Separando data de hora
aux_0 = df_train['date'].unique()
day = 1

for x in range(len(df_train)):
    #Extraindo data
    aux_1 = aux_0[x-1]
    date = aux_1[0:10]

    df_train.at[x, 'date'] = date

    #Extraindo mês
    aux_1 = aux_0[x-1]
    month = aux_1[5:7]

    df_train.at[x, 'month'] = month

    #Extraindo hora
    aux_1 = aux_0[x]
    hours = aux_1[10:13]

    df_train.at[x, 'hours'] = hours

In [244]:
df_train.head(50)

,date,month,hours,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-12-19,12,00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01,01,01,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01,01,02,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01,01,03,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01,01,04,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01,01,05,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01,01,06,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01,01,07,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01,01,08,1,0,0,1,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01,01,09,1,0,0,1,13.12,17.425,76,0.0000,8,6,14


In [245]:
#Alterando os dados object
df_treino = pd.get_dummies(df_train, columns = ['holiday', 'workingday', 'season', 'weather', 'month', 'hours'], prefix = ['holiday', 'workingday', 'season', 'weather', 'month', 'hours'])

In [246]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 56 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          10886 non-null  object 
 1   temp          10886 non-null  float64
 2   atemp         10886 non-null  float64
 3   humidity      10886 non-null  int64  
 4   windspeed     10886 non-null  float64
 5   casual        10886 non-null  int64  
 6   registered    10886 non-null  int64  
 7   count         10886 non-null  int64  
 8   holiday_0     10886 non-null  uint8  
 9   holiday_1     10886 non-null  uint8  
 10  workingday_0  10886 non-null  uint8  
 11  workingday_1  10886 non-null  uint8  
 12  season_1      10886 non-null  uint8  
 13  season_2      10886 non-null  uint8  
 14  season_3      10886 non-null  uint8  
 15  season_4      10886 non-null  uint8  
 16  weather_1     10886 non-null  uint8  
 17  weather_2     10886 non-null  uint8  
 18  weather_3     10886 non-nu

### Organização das Features e Target

In [257]:
X = df_treino.drop(columns = ['date', 'count', 'windspeed', 'casual', 'registered'])
X.head()

,temp,atemp,humidity,holiday_0,holiday_1,workingday_0,workingday_1,season_1,season_2,season_3,...,hours_ 14,hours_ 15,hours_ 16,hours_ 17,hours_ 18,hours_ 19,hours_ 20,hours_ 21,hours_ 22,hours_ 23
0,9.84,14.395,81,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.02,13.635,80,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9.02,13.635,80,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9.84,14.395,75,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9.84,14.395,75,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
y = df_treino['count']
y.shape

(10886,)

### Cross Validation - KFOLD

In [259]:
clf = DecisionTreeRegressor()

In [277]:
scores = cross_val_score(clf, X, y, cv = 100)

In [278]:
scores

array([-2.88339315, -0.1443479 ,  0.09657892, -0.89891367, -0.02308086,
       -1.05991533, -1.59017382, -2.09351735, -0.45699525, -2.99667861,
       -1.77458369, -1.62246086, -0.35398827, -0.52863903, -0.19066735,
       -1.27366716,  0.18168692,  0.28785672,  0.45750541,  0.36859383,
        0.51312191,  0.39501673,  0.54680589,  0.40901607,  0.54941455,
        0.55707198,  0.56751147,  0.54173168,  0.62232611,  0.4845252 ,
        0.20259788,  0.45994524,  0.42998991,  0.53838393,  0.13378921,
        0.6772408 ,  0.37038682,  0.23730241,  0.57972316,  0.38849721,
       -0.15728614,  0.70630113,  0.64053354,  0.58708321,  0.17538437,
        0.33782789,  0.29066513,  0.20872008,  0.13740712,  0.40637998,
        0.64525045,  0.49853786,  0.4671672 ,  0.5668424 ,  0.70475254,
        0.58761314,  0.69328877,  0.5726    ,  0.55763743,  0.4991774 ,
        0.68663874,  0.51470398,  0.66209107,  0.54225361,  0.63330096,
        0.67076679,  0.59757072,  0.76838267,  0.71513033,  0.66

In [279]:
scores.mean()

0.29901785651476165